# 👩‍⚕️ Lecture 5 – Data 100, Spring 2026

[Acknowledgments Page](https://ds100.org/sp26/acks/)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
#plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# Use 5 decimal places instead of scientific notation in pandas
pd.set_option('display.float_format', '{:.5f}'.format)

# Silence some spurious seaborn warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Loading Data

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(3)

# 💁 Gender-Neutrality in Names

In [ ]:
def gender_neutrality(series):
    """Given a series of counts for different genders, computes how gender-neutral the name is"""
    # How many different ways can you find to calculate the same metric?
    return 2 * (0.5 - np.abs(0.5 - series.iloc[0] / series.sum()))

In [ ]:
neutralities_06 = (
    babynames[babynames['Year'] == 2006]
    .groupby('Name')[['Count']].agg(gender_neutrality)
    .rename(columns={'Count': 'Gender neutrality'})
    .reset_index()
)
neutralities_06.sort_values('Gender neutrality', ascending=False).head(10)

In [ ]:
# Gender-neutrality of each name within each year
neut_yearly = (
    babynames.groupby(['Name', 'Year'])[['Count']].agg(gender_neutrality)
    .reset_index()
    .rename(columns={'Count': 'Gender neutrality'})
)
neut_yearly



In [ ]:
most_neutral_name_by_year = (
    neut_yearly
    .sort_values('Gender neutrality', ascending=False)
    .groupby('Year').head(1)
    .sort_values('Year')
)
most_neutral_name_by_year

This gives us the most gender-neutral name for each year, but we can see by inspecting it that many of these names are rare. What if we're only interested in more common names?

We need to use the counts.

In [ ]:
yearly_name_counts = babynames.groupby(['Name', 'Year'])[['Count']].sum().reset_index()
yearly_name_counts.head(4)

In [ ]:
counts_and_neutrality = pd.merge(
    yearly_name_counts, neut_yearly, 
    left_on=['Name', 'Year'], right_on=['Name', 'Year']
)
counts_and_neutrality

In [ ]:
most_neutral_name_by_year = (
    counts_and_neutrality[counts_and_neutrality['Count'] > 500]
    .sort_values('Gender neutrality', ascending=False)
    .groupby('Year').head(1)
    .sort_values('Year')
)
most_neutral_name_by_year

In [ ]:
sns.lineplot(most_neutral_name_by_year, x='Year', y='Gender neutrality')

In [ ]:
# What happened around 1940?
most_neutral_name_by_year[
    (most_neutral_name_by_year['Year'] > 1935) & 
    (most_neutral_name_by_year['Year'] < 1950)
]

# 🦠 Flu in the United States

What can we say about flu in the United States?

## 📖 Reading CSVs

The flu data contains several CSV files, located in `data/flu/`. We can explore them in many ways:

1. Using the JupyterLab explorer tool (read-only!).
2. Opening the CSV in DataHub, or Excel, or Google Sheets, etc.
3. Opening the file in Python with `open()`
4. With `pandas`, using `pd.read_csv()`

<br>


---

## 🧭 Methods 1 and 2: Play with the data in the JupyterLab Explorer and DataHub
 To solidify the idea of a CSV as **rectangular data** (i.e., tabular data) stored as comma-separated values, let's start with the first two methods.  

 **1. Use the file browser in JupyterLab to locate one of the CSVs in `data/flu.csv`, and double-click on it.**

  **2. Right-click on the CSV in the file browser. Select `Open With` --> `Editor`. But, don't make any changes to the file!**

<br>

---

## 🐍 Method 3: Play with the data in Python

Next, we will load in the data as a Python file object and inspect a couple lines. 

With the code below, we can check out the first four lines of the CSV:

In [ ]:
# Open the FLU ILINet CSV, and print the first four lines
with open("data/flu/ILINet.csv", "r") as f:
    for i, row in enumerate(f):
        print(row)
        if i >= 3: break

As expected, most of the lines are comma-separated values. But what's up with the first line?

> Why are there blank lines between each line of the CSV file?
>
> You may recall that line breaks in text files are encoded with the special newline character `\n`. 
> 
> Python's `print()` function prints each line, interpreting the `\n` at the end of each line as a newline, **and also adds an additional newline**.

We can use the `repr()` ("representation") function to return the raw string representation of each line (i.e., all special characters will be visible).

- In other words, `print()` will not interpret `\n` as a newline. Instead, it will literally print `\n`.

- Note, though, `print()` adds a newline each time it is called. Otherwise, we would have one long string below instead of four lines.

In [ ]:
# Open the TB case count CSV, and print the raw representation of
# the first four lines
with open("data/flu/ILINet.csv", "r") as f:
    for i, row in enumerate(f):
        print(repr(row)) # print raw strings
        if i >= 3: break

## 🐼 Method 4: Play with the data using `pandas`

Time to use our favorite Data 100 approach: `pandas`.

In [ ]:
ili = pd.read_csv("data/flu/ILINet.csv")
ili

Why is there only one column? And why are most of the data values bolded?

We're ready to wrangle the data! 

A reasonable first step is to **identify the row with the right header** (i.e., the row with the column names). 

The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has a convenient `header` parameter for specifying the index of the row you want to use as the header:

In [ ]:
# Try again, this time telling pandas that the header is actually on the second line
ili = pd.read_csv("data/flu/ILINet.csv", header=1)
ili.head()

### 🔎 Granularity of records

What is the granularity of each record in our flu dataset? Region? Week? Age range?

The answer is that each row represents a unique combination of week, year, and region.

We should probably start by making some **visualizations**: a picture is always worth a thousand words.

## 📈 Time to visualize! (Almost)

### Preprocessing and column manipulation

...except we can't just yet. Since we have one row per year/week, we probably want to use a line graph. But, we need one column we can use for the x-axis in our graph. How do we convert a year column + week column into a single datetime column?

In [ ]:
# pd.to_datetime is a very useful function: 
# it's worth remembering and being able to look up how the format strings work
ili['week_start'] = pd.to_datetime(
    (ili['YEAR'] * 100 + ili['WEEK']).astype(str) + '0', 
    format='%Y%W%w'
)
ili.sample(3)

We can access different attributes of the new column using the `.dt` accessor / attribute:

In [ ]:
ili['week_start'].dt.year.head()

What is the type of this new column?

In [ ]:
ili['week_start'].dtype

`ns` above stands for nanoseconds.

- `<M8` refers to the Numpy type `datetime64`

Under the hood, datetimes in Pandas are integers representing the number of **nanoseconds** since 1/1/1970 UTC.

### Visualization

Don't worry too much about the exact code that generates the graph: we'll talk more about it next week.

In [ ]:
# Visualize number of cases for babies and small children
f, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.lineplot(ili, x='week_start', y='AGE 0-4', hue='REGION', ax=ax);

In [ ]:
# Visualize overall prevalence of flu
f, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.lineplot(ili, x='week_start', y='% WEIGHTED ILI', hue='REGION', ax=ax);

At this point, stop and think about any interesting observations or questions you have about this data.

<br/><br/>

---

**Instructor note: Return to the slides!**


---
<br/><br/>



# Load vaccination data

This dataframe contains vaccination numbers by each flu season (starting in July and ending the following June).

Take a close look at the output, and make sure you understand what happens between 2023-06-01 and 2023-07-01!

In [ ]:
vax = pd.read_csv('data/flu/monthly_child_flu_vaccination.csv')
vax['month_dt'] = pd.to_datetime(vax['month_dt'])
vax['rate'] = vax['Numerator'] / vax['Population']
vax.head(14)

<br/><br/>

---

# 👥 Joining ILI counts with vaccination data

Time to `merge` our datasets (i.e., join them)! 

In [ ]:
# Show the three tables that we are going to join.
# To keep things simple, let's just look at the last two rows of each df.
display(ili.tail(2))
display(vax.tail(2))


We need to have a month column in ILI so that we can join them. How do we convert our `week_start` column to the corresponding month? This is a great time to do a Google search and/or ask an LLM!

In [ ]:
ili["month"] = ili["week_start"].dt.to_period("M").dt.to_timestamp()

In [ ]:
ili_vax = ili.merge(
    vax, 
    left_on=['month', 'REGION'], 
    right_on=['month_dt', 'HHS Region']
)
ili_vax.head()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 7))
sns.scatterplot(ili_vax, x='ILITOTAL', y='rate', alpha=0.3, hue='HHS Region', ax=ax)

What are some limitations of this graph?